# Azure AI Search용 RAG 퀵스타트

이 퀵스타트는 RAG(검색 증강 생성) 시나리오를 위한 쿼리 예시를 제공합니다. Azure AI Search의 검색 인덱스에서 근거 데이터를 활용하여 챗 경험을 구현하는 방법을 보여줍니다.

이 연습을 간단하게 유지하고 쿼리 정의에 집중하기 위해 몇 가지 단계를 생략했습니다:

- **hotels-sample-index**를 사용합니다. 이 인덱스는 몇 분 만에 생성할 수 있으며, 모든 검색 서비스 티어에서 실행됩니다. 이 인덱스는 내장 샘플 데이터를 사용하여 마법사로 생성됩니다.

- 벡터를 생략하여 청킹(chunking)과 임베딩 단계를 건너뜁니다. 인덱스에는 일반 텍스트만 포함되어 있습니다.

비벡터 인덱스는 RAG 패턴에 이상적이지 않지만, 더 간단한 예시를 제공합니다.

Azure AI Search의 쿼리를 Azure OpenAI의 챗 컴플리션 모델과 통합하는 기본 원리를 이해하면, 벡터 필드와 벡터/하이브리드 쿼리를 추가하여 경험을 확장할 수 있습니다. 그 단계에 대해서는 [phi-chat Python 코드 예제](https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/phi-chat/phi-chat.ipynb)를 참고하세요.

이 예제는 [빠른 시작: Azure AI Search의 근거 데이터를 활용한 생성형 검색(RAG)](https://learn.microsoft.com/azure/search/search-get-started-rag)에 완전히 문서화되어 있습니다. readme보다 더 많은 안내가 필요하다면 해당 문서를 참고하세요.


## 사전 준비 사항

- [Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource)

  - 사용하려는 챗 컴플리션 모델(gpt-4o, gpt-4o-mini 또는 동등 모델)을 지원하는 [지역 선택](https://learn.microsoft.com/azure/ai-services/openai/concepts/models?tabs=global-standard%2Cstandard-chat-completions#global-standard-model-availability)
  - Azure AI Foundry에서 [챗 컴플리션 모델 배포](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models-openai) 또는 [다른 방법 사용](https://learn.microsoft.com/azure/ai-services/openai/how-to/working-with-models?tabs=powershell)

- [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal)

  - Basic 티어 이상 권장
  - 시맨틱 랭킹 활성화
  - 역할 기반 액세스 제어(RBAC) 활성화
  - Azure AI Search에 시스템 ID 활성화
  
배포된 모델의 이름과 두 Azure 리소스의 엔드포인트를 반드시 알아두세요. 이후 단계에서 이 정보를 사용합니다.

## 액세스 구성

이 퀵스타트는 Microsoft Entra ID 및 역할 할당을 통한 인증 및 권한 부여를 가정합니다. 또한 로컬 장치에서 코드를 실행한다고 가정합니다.

1. Azure AI Search에서 샘플 호텔 인덱스를 생성, 로드, 쿼리하려면 **Search Index Data Contributor** 및 **Search Service Contributor** 역할이 필요합니다. 호텔 인덱스가 이미 있다면 **Search Index Data Reader**만 있으면 됩니다.

2. 쿼리 및 검색 결과를 Azure OpenAI로 보내려면 Azure OpenAI에서 **Cognitive Services OpenAI User** 권한이 필요합니다.

## 샘플 인덱스 생성

이 퀵스타트는 hotels-sample-index를 사용하며, [이 빠른 시작](https://learn.microsoft.com/azure/search/search-get-started-portal)을 통해 몇 분 만에 생성할 수 있습니다.

인덱스가 생성되면 Azure 포털에서 다음과 같이 시맨틱 구성을 적용하세요:

이제 Azure 리소스, 인덱스, 모델이 준비되었으니, 스크립트를 실행하여 인덱스와 대화할 수 있습니다.

## 가상 환경 만들기

의존성을 격리하여 설치할 수 있도록 가상 환경을 만듭니다.

1. Visual Studio Code에서 Quickstart-RAG.ipynb가 포함된 폴더를 엽니다.

2. Ctrl-shift-P를 눌러 명령 팔레트를 열고, "Python: Create Environment"를 검색한 후 `Venv`를 선택하여 현재 작업 공간에 가상 환경을 만듭니다.

3. Quickstart-RAG\requirements.txt를 의존성 파일로 선택합니다.

환경 생성에는 몇 분이 소요될 수 있습니다. 환경이 준비되면 다음 단계로 진행하세요.

## Azure에 로그인

연결을 위해 Microsoft Entra ID와 역할 할당을 사용합니다. Azure AI Search 및 Azure OpenAI와 동일한 테넌트 및 구독에 로그인되어 있는지 확인하세요. 현재 속성 확인, 속성 변경, 로그인 등은 Azure CLI를 사용할 수 있습니다. 자세한 내용은 [키 없이 연결](https://learn.microsoft.com/azure/search/search-get-started-rbac)을 참고하세요.

아래 명령어를 순서대로 실행하세요.

```
az account show

az account set --subscription <여기에 구독 ID 입력>

az login --tenant <여기에 테넌트 ID 입력>
```

이제 로컬 장치에서 Azure에 로그인된 상태입니다.

## 코드 실행

In [1]:
# Package install for quickstart
! pip install -r requirements.txt --quiet

In [2]:
# Set endpoints and deployment model (provide the name of the deployment)
AZURE_SEARCH_SERVICE: str = "PUT YOUR SEARCH SERVICE ENDPOINT HERE"
AZURE_OPENAI_ACCOUNT: str = "PUT YOUR AZURE OPENAI ENDPOINT HERE"
AZURE_DEPLOYMENT_MODEL: str = "gpt-4o"

## 기본 RAG 쿼리

아래 코드는 검색 인덱스의 문자열 필드 및 문자열 컬렉션에 쿼리를 설정하고, 결과를 챗 컴플리션 모델로 보내는 방법을 보여줍니다. 사용자에게 반환되는 응답은 챗 컴플리션 모델의 결과입니다.

In [ ]:
# Set up the query for generating responses
from azure.identity import DefaultAzureCredential
from azure.identity import get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
openai_client = AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ACCOUNT,
    azure_ad_token_provider=token_provider
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE,
    index_name="hotels-sample-index",
    credential=credential
)

# This prompt provides instructions to the model
GROUNDED_PROMPT="""
당신은 액티비티와 편의시설에 따라 호텔을 추천하는 친절한 도우미입니다.
아래에 제공된 출처만을 사용하여 친절하고 간결한 글머리 기호 방식으로 쿼리에 답변하세요.
아래 출처 목록에 나열된 사실만으로 답변하세요.
아래에 정보가 충분하지 않은 경우 모른다고 답하세요.
아래 출처를 사용하지 않은 답변은 작성하지 마세요.
Query: {query}
Sources:\n{sources}
"""

# Query is the question being asked. It's sent to the search engine and the LLM.
query="무료 조식이 제공되는 호텔을 추천해 주시겠어요?"

# Search results are created by the search client.
# Search results are composed of the top 5 results and the fields selected from the search index.
# Search results include the top 5 matches to your query.
search_results = search_client.search(
    search_text=query,
    top=5,
    select="Description,HotelName,Tags",
    query_type="semantic"
)
sources_formatted = "\n".join([f'{document["HotelName"]}:{document["Description"]}:{document["Tags"]}' for document in search_results])

# Send the search results and the query to the LLM to generate a response based on the prompt.
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=AZURE_DEPLOYMENT_MODEL
)

# Here is the response from the chat model.
print(response.choices[0].message.content)

결과 대신 인증 오류가 발생한다면:

+ 역할 할당을 방금 활성화했다면, 몇 분 후 다시 시도하세요. 역할 할당이 적용되기까지 시간이 걸릴 수 있습니다.

+ Azure AI Search에서 [RBAC를 활성화](https://learn.microsoft.com/azure/search/search-security-enable-roles?tabs=config-svc-portal%2Cdisable-keys-portal)했는지 확인하세요. **Forbidden** 상태 메시지는 RBAC가 활성화되지 않았음을 의미할 수 있습니다.

+ 401 오류가 발생하면 역할 할당을 다시 확인하세요. [키 없이 연결](https://learn.microsoft.com/azure/search/search-get-started-rbac) 단계도 검토하세요.

+ 방화벽 설정을 확인하세요. 이 퀵스타트는 공용 네트워크 액세스를 가정합니다. 방화벽이 있다면 장치 및 서비스 간 연결을 허용하는 규칙을 추가해야 합니다. 자세한 내용은 [네트워크 액세스 구성](https://learn.microsoft.com/azure/search/service-configure-firewall)을 참고하세요.

+ 추가 디버깅 안내는 퀵스타트 문서의 [문제 해결 섹션](https://learn.microsoft.com/azure/search/search-get-started-rag#troubleshooting-errors)을 참고하세요.

## 복잡한 RAG 쿼리

복합 타입 및 컬렉션을 RAG 패턴에서 사용하려면 결과를 JSON으로 변환한 후 챗 컴플리션 모델로 전달하세요. 이 쿼리는 기본 쿼리와 유사하지만, "Address" 복합 타입과 "Rooms" 복합 컬렉션의 필드를 포함합니다. 인덱스의 단순 텍스트 필드가 아닌 부분에서 데이터를 가져오는 방법을 보여줍니다.

In [ ]:
import json

# Query is the question being asked. It's sent to the search engine and the LLM.
query="무료 조식을 제공하는 호텔을 몇 군데 추천해 주시겠어요? 호텔 설명, 주소, 태그, 4인 1실의 객실 요금을 알려주세요."

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
selected_fields = ["HotelName","Description","Address","Rooms","Tags"]
search_results = search_client.search(
    search_text=query,
    top=5,
    select=selected_fields,
    query_type="semantic"
)
sources_filtered = [{field: result[field] for field in selected_fields} for result in search_results]
sources_formatted = "\n".join([json.dumps(source) for source in sources_filtered])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=AZURE_DEPLOYMENT_MODEL
)

print(response.choices[0].message.content)